In [ ]:
# 자연어 생성 모델의 경우 어떤 방식으로 생성하느냐에 따라 성능 차이가 커짐
# 자연어 생성 모델의 생성 방법들에 대해 이해하고 실제 적용 및 응용

# 모델 불러오기

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2

Detected operating system as Ubuntu/jammy.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.4.13
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages will be upgraded:
  git-lfs
1 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Need to get 8,489 kB of archives.
After this operation, 7,671 kB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu jammy/main amd64 git-lfs amd64 3.6.1 [8,489 kB]
Fetched 8,489 kB in 0s (26.9 MB/s)
(Reading database ... 125048 files and directories c

In [ ]:
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel

tokenizer = SentencePieceBPETokenizer("/content/kogpt2/vocab.json", "/content/kogpt2/merges.txt")

config = GPT2Config(vocab_size=50000)
config.pad_token_id = tokenizer.token_to_id('<pad>')
model = GPT2LMHeadModel(config)

model_dir = '/content/kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location='cuda'), strict=False)
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50000, bias=False)
)

In [ ]:
model.eval()
tokenizer.add_special_tokens(["<s>", "</s>"])

2

# Greedy Search
 단순히 가장 높은 확률을 가진 단어를 다음 단어로 선택

In [ ]:
def tokenizing(text):
    return torch.tensor(tokenizer.encode('<s> '+text, add_special_tokens=False).ids).unsqueeze(0).to('cuda')

In [ ]:
input_ids = tokenizing("이순신은 조선 중기의 무신이다.")

# generate text until the output length (which includes the context length) reaches 100
# 생성 모델은 generate 함수를 통해 다음 token을 생성

greedy_output = model.generate(input_ids, max_length=100)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output.tolist()[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
<s> 이순신은 조선 중기의 무신이다.</s><s> 이 때문에 '이순신'은 '이순신'의 '이순신'으로, '이순신'은 '이순신'으로 각각 불린다.</s><s> '이순신'은 '이순신'으로, '이순신'은 '이순신'으로 각각 불린다.</s><s> '이순신'은 '이순신'으로, '이순신'은 '


In [ ]:
# 생성된 단어의 문맥은 합리적이지만 출력을 살펴보면 비슷한 단어가 계속 반복되는 문제가 보임

# Greedy 탐색의 주요 단점은 낮은 확률 단어 이후에 나올수 있는 더 높은 확률의 단어를 놓친다는 점

# Beam Search
 각 단계에서 가장 확률이 높은 단어들의 num beams(정해진 개수)를 유지하고 결국 전체 확률이 가장 높은 순서를 선택하는 방법

In [ ]:
beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output.tolist()[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
<s> 이순신은 조선 중기의 무신이다.</s><s> (서울= 뉴스와이어) 2012년 04월 05일 -- 국토해양부(장관 : 권도엽)는 2012년 4월 10일, 국토해양부(장관 : 권도엽)는


In [ ]:
# 동일한 단어가 계속해서 반복되는 현상이 동일하게 발생

# 이러한 현상은 일반적인 언어생성 모델에서 나타나는 공통적인 문

# 단순한 해결법은 n-gram 패널티를 도입하는 것

# "no_repeat_ngram_size=2" 로 설정을 하면 동일 단어가 2번 이상 나타나는 것을 방지할 수 있음

In [ ]:
beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    early_stopping=True,
    no_repeat_ngram_size=2
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output.tolist()[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
<s> 이순신은 조선 중기의 무신이다.</s><s> 그 후, 이조참의(吏曹參議)가 되고, 의정부영의정(議政府儀政)을 거쳐, 영의정이 되고 의정부좌찬성에 이르렀다.


In [ ]:
# Beam 탐색의 또 다른 특징으로 여러 후보들을 생성할 수 있음

# "num_return_sequences=5"라는 설정을 하면, 총 5개의 생성 결과를 받아 봄

# 단 num_return_sequences는 당연히 num_beams보다 작거나 같은 숫자이어야함

In [ ]:
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output.tolist(), skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: <s> 이순신은 조선 중기의 무신이다.</s><s> 그 후, 그 해 11월 16일(음력 9월 15일)에 향년 60세를 일기로 사망하였다. &lt;삼국사기&gt;에 따르면, 그는 《
1: <s> 이순신은 조선 중기의 무신이다.</s><s> 그 후, 그 해 11월 16일(음력 9월 15일)에 향년 60세를 일기로 사망하였다. &lt;삼국사기&gt;에 따르면, 신라 경
2: <s> 이순신은 조선 중기의 무신이다.</s><s> 그 후, 그 해 11월 16일(음력 9월 15일)에 향년 60세를 일기로 사망하였다. &lt;삼국사기&gt;에 따르면, 신라에
3: <s> 이순신은 조선 중기의 무신이다.</s><s> 그 후, 그 해 11월 16일(음력 9월 15일)에 향년 60세를 일기로 사망하였다. &lt;삼국사기&gt;에 따르면, 그의 아들
4: <s> 이순신은 조선 중기의 무신이다.</s><s> 그 후, 그 해 11월 16일(음력 9월 15일)에 향년 60세를 일기로 사망하였다. &lt;삼국사기&gt;에 따르면, 그가 죽


In [ ]:
# 개방형 생성에서는 Beam 탐색이 최선이 아닐 수 있는 이유

# 문장 생성 길이가 예측 가능한 태스크에서 잘 동작할 수 있지만, 대화 같이 길이가 유동적인 태스크에서는 잘 작동하지 않음.
# 반복 생성 문제에 취약
# 고품질 인간 언어는 높은 확률의 다음 단어 분포를 따르지 않음

# Sampling

사람의 언어를 보면 무조건 높은 확률을 선택하지 않기 때문에, 생성에 랜덤성을 부여할 필요가 있음

그래서 제시된 방법이 Sampling

가장 기본적인 형태의 Sampling은 조건부 확률 분포에 따라 다음 단어를 무작위로 선택하는 것을 의미

In [ ]:
sample_output = model.generate(
    input_ids,
    do_sample=True, # 완전 random sampling
    max_length=50,
    top_k=0 # w/o top_k 추출 Top-K sampling을 비활성화
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output.tolist()[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
<s> 이순신은 조선 중기의 무신이다.</s><s> 정보는 ‘왕의 정원’이라는 레이건 일가의 못된 마음을 바로잡는 데 공로를 세웠다는 평을 받는다.</s><s> 이 문서에서 그는 그는 탬파 대국회로 가민(嘉


In [ ]:
# 본문은 괜찮아 보이지만, 자세히 보면 일관성이 거의 없음

# 이 문제를 보완하기 위한 한가지 트릭은 소프트맥스의 temperature를 낮추어 분포를 더 선명하게 만드는 것

# 높은 확률의 단어의 가능성은 증가시키고, 낮은 확률의 단어 가능성은 감소시키는 효과를 줄 수 있음

In [ ]:
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=0,
    temperature=0.7
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output.tolist()[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
<s> 이순신은 조선 중기의 무신이다.</s><s> 영국왕립군</s><s> 영국왕립군()은 영국의 국왕을 보좌하는 국체로, 영국의 행정기구로, 영국 제2대 국왕인 영국의 엘리자베스 2세 여왕(재위


In [ ]:
# temperature를 적용해 분포가 덜 랜덤해 졌지만, 0으로 설정해버리면 Greedy 탐색과 동일해져버림

# Top-K sampling

K=6으로 설정을 하면, Sampling pool이 확률이 가장 높은 6개의 단어로 제한.

첫 step에서는 0.68로 일부분에서만 디코딩 되고, 두번째 step에서는 0.99까지 올라감

하지만, 두번째 샘플링에서 "not", "the", "small", "told"와 같이 이상한 후보들이 잘 제거됨

이러한 방식으로 일관성 있으면서, 랜덤성을 부여하는게 가능

In [ ]:
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output.tolist()[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
<s> 이순신은 조선 중기의 무신이다.</s><s> 하지만 3명의 주사가 각각 1대 1로 대치된 상황으로, 2교시 5분에 두 개의 주사가 1대 1로 대치되다가 4교시 10분(각 1대


# Top-p (nucleus) sampling

확률이 가장높은 K개의 단어가 아니라, 누적확률이 P를 초과하는 최소한의 단어 집합에서 샘플링을 진행하는 방식

이렇게 하면 필터링된 단어수 또한 동적으로 변화가 가능

In [ ]:
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_p=0.92,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output.tolist()[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
<s> 이순신은 조선 중기의 무신이다.</s><s> 박노자 본명 '논어'는 줄임말 '문(文)'이 있지, 듣기만 해보면 속담만쓰게 돼 있다.</s><s> 청중 앞에서 참


In [ ]:
# 이론적으로는 top-p가 top-k보다 더 성능이 좋을 것 같지만, 실제로는 크게 다르지 않음

# top-p와 top-k는 동시에 사용할 수도 있음

In [ ]:
beam_outputs = model.generate(
    input_ids,
    do_sample=True,
    max_length=100,
    top_k=50,
    top_p=0.95,
    num_return_sequences=5
)

print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output.tolist(), skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: <s> 이순신은 조선 중기의 무신이다.</s><s> 그는 《신증동국여지승람》, 《경국대전》, 《유월지절》, 《조선지신론》 3권을 통해 유학의 개념을 체계화하고, 그 이론을 토대로 하여, 조선 성리학의 중요한 학술적 근거로 삼은 저작이다.</s><s> 그는 《유월지절》, 《경국대전》, 《유월지절》 등을 통해 성리학의 근본개념, 방법론, 학문, 그리고 성리학의
1: <s> 이순신은 조선 중기의 무신이다.</s><s> ⊙ 제15대 서울지방법원장 이상욱(사법시험 20회)</s><s> ▶ 더 빠르고 편리하게 만나는 실시간 모바일 뉴스</s><s> 최필립, 이사장 사퇴 요구..</s><s> 이사장직 사퇴 의사표명 - 중앙일보 뉴스</s><s> 최필립, 이사장직 사퇴 요구..</s><s> 이사장직 사퇴 의사표명 - 중앙일보 뉴스</s><s> 5일 정치권에 따르면 최필립 이사장은 최근 언론 인터뷰를 통해 자신의 이사장으로 취임하는 것이 한국 사회의 발전에
2: <s> 이순신은 조선 중기의 무신이다.</s><s> 제16장: <남정>에서 말하는 정사는(정) 1.</s><s> 조선은 18년(1836) 7월 2일(조선 고종 1년 3월 2일)에 일본과의 외교관계를 수립하였다.</s><s> 제174년(조선 고종 23년 2월 1일) 10일(조선 고종 4년 12월 16일)까지 일본에 파견되어 있던 일본군 5천 명과 일본으로부터 외교적인
3: <s> 이순신은 조선 중기의 무신이다.</s><s> 이 중 한사람은 "김 선생이 이 땅의 민주주의와 국민주권을 지키기 위해 헌신하신 것을 잊지 않고 있다"면서 "역사의 물길을 따라 걸어갔을 때, 이 땅의 자유와 평화가 더 이상 헛되지 않을 것이라고 확신한다"고 말했다.</s><s> 김 선생은 광복 후 이 땅의 민주화를 위해 헌신하다가 1953년 11월 서

In [ ]:
# ad-hoc decoding방법에 따르면 Top-p와 Top-k sampling은 기존 Greedy-, Beam search 보다 개방형 언어생성에서 더 유창한 문장을 생성하는 것으로 보임
# 그러나 Top-p와 Top-k sampling 또한 Greedy-, Beam search 처럼 반복적 Word sequencd에 대한 문제가 발생
# 논문에 따르면 Beam search가 모델 트레이닝 목적함수를 잘 조정한다면 Top-p보다 더 유찬한 텍스트를 생선한다는 연구도 있음
# 개방형 언어생성은 빠르게 발전하는 분야이며, 무엇이 적합하다고 단정할 수 없으므로 특정 사용 사례에서 가장 잘 작동하는 방법이 무엇인지 고려
# 언어별, 모델별로 최적의 생성 방식을 찾아내야 함
